In [23]:
! pip install requests
! pip install beautifulsoup4
! pip install lxml
! pip install pandas
! pip install sklearn

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import time
import pandas as pd
from sklearn.cluster import DBSCAN

In [5]:
stars = input("스타수 입력 : ")
url = "https://github.com/search?p=1&q=stars%3A%3E{}&type=Repositories".format(stars)

def crawling_func(url):
    print(url)
    try:
        res = requests.get(url)
        res.raise_for_status()
        soup = BeautifulSoup(res.text,"lxml")
        p = soup.find_all("a",attrs={"class":"v-align-middle"}) 
    except:
        time.sleep(5)
        crawling_func(url) # 오류를 대비하기위해 재귀함수 호출
        pass
    finally:
        return p
    
topic_ads = []
pages = int(input("검색할 페이지 수(Ex : 10) : "))
print("{}개 이상의 star수를 가진 Repository의 상위 {}페이지를 크롤링합니다. ".format(stars,pages))
print()
for i in range(1,pages+1):# x 페이지까지 탐색
    url = "https://github.com/search?p={}&q=stars%3A%3E10000&type=Repositories".format(i) # 페이지 formatting
    time.sleep(15)  # 10초를 쉬어도 오류가 떴었음
    for j in crawling_func(url):
        topic_ads.append(j.get_text())
        print(j.get_text())

        
topics_dic = {}
topics_list = []
for ad in topic_ads:
    url_topic = "https://github.com/" + ad
    res_topic = requests.get(url_topic)
    res_topic.raise_for_status()
    soup_topic = BeautifulSoup(res_topic.text,"lxml")
    
    topic = soup_topic.find("div",attrs={"class":"BorderGrid-cell"}).find_all("a",attrs={"class":"topic-tag topic-tag-link"})
    project_topics=[i.get_text().replace("\n","").replace("\t","").strip() for i in topic]
    
    # 스타 수(북마크 수, 인기도)
    star_num = soup_topic.find("div",attrs = {"class":"BorderGrid-cell"}).find("strong").get_text()
    star_num = star_num.replace('\t', '').replace('\n', '').strip()
    topics_list.append([ad,project_topics,star_num])
    
    for t in project_topics:
        if t in topics_dic:
            topics_dic[t] += 1
        else:
            topics_dic[t] = 1

# value로 내림차순 정렬
topics_dic= sorted(topics_dic.items(), key=lambda x: x[1], reverse=True)
df_topic2 = pd.DataFrame(topics_list,columns=['project_name','topic_keyword','star_number'])
# 엑셀로 저장하기

df_topic2.to_excel('Topics_stars{}_project_keyword.xlsx'.format(stars),index=False)
print()
print("star 수가 {} 개 이상인 프로젝트 상위 {}페이지에대한 Data가 저장되었습니다.".format(stars,pages))
print("종료 .")


스타수 입력 : 10
검색할 페이지 수(Ex : 10) : 3
10개 이상의 star수를 가진 Repository의 상위 3페이지를 크롤링합니다. 

https://github.com/search?p=1&q=stars%3A%3E10000&type=Repositories
freeCodeCamp/freeCodeCamp
996icu/996.ICU
EbookFoundation/free-programming-books
jwasham/coding-interview-university
sindresorhus/awesome
kamranahmedse/developer-roadmap
public-apis/public-apis
vuejs/vue
facebook/react
donnemartin/system-design-primer
https://github.com/search?p=2&q=stars%3A%3E10000&type=Repositories
tensorflow/tensorflow
twbs/bootstrap
getify/You-Dont-Know-JS
CyC2018/CS-Notes
codecrafters-io/build-your-own-x
ohmyzsh/ohmyzsh
trekhleb/javascript-algorithms
flutter/flutter
TheAlgorithms/Python
github/gitignore
https://github.com/search?p=3&q=stars%3A%3E10000&type=Repositories
torvalds/linux
microsoft/vscode
vinta/awesome-python
airbnb/javascript
Snailclimb/JavaGuide
jackfrued/Python-100-Days
ossu/computer-science
ytdl-org/youtube-dl
labuladong/fucking-algorithm
jlevy/the-art-of-command-line

star 수가 10 개 이상인 프로젝트 상위 3페이지에대한

In [6]:
url_topic = "https://github.com/" + "mui/material-ui"
res_topic = requests.get(url_topic)
res_topic.raise_for_status()
soup_topic = BeautifulSoup(res_topic.text,"lxml")
soup_topic
star_num = soup_topic.find("div",attrs = {"class":"BorderGrid-cell"}).find("strong")
star_num.get_text()
# star_num = star_num.replace('\t', '').replace('\n', '').strip()

'79.5k'

# DBSCAN Clustering

In [7]:
# 비슷한 토픽끼리는 하나의 칼럼이 될 수 있도록 일일히 비슷한 토픽끼리 묶는 작업을 하였음
target = ['machinelearning','package','zsh','zerossl','youtube','xcode','x64','wysiwyg','workflow','windows','weixin','wechat','web','vue',
         'vpn','vision','vim','vedio','version','util','ui','typescript','twitter','touch','test','terminal','tensorflow',
         'telegram','task','system','swift','swagger','svg','style','static','state','ssh','sql','spring','sort','software',
         'snippets','server','security','search','scikit-learn','rfc','rest','redis','react','queue','pytorch','python','protoc','progress',
         'programming','posix','php','performance','openwrt','note','nodejs','neural','nlp','monitoring','microservice','material','macos','linux',
          'leetcode','language','kotlin','json','javascript','ios','interview','http','html','headless','graphql','git','frontend','font','flutter',
          'firefox','file','face-swap','docker','distributed','design','deep-learning','data','css','cloud','chrome','automation','api','angular',
          'android','algorithm','pull-request', 'public','pixi','open-source','object-detection','multi-platform','mini','low-code','localstorage','loading',
          'kubernetes','koa','image','humans','game','free','detection','drag','document','devops','dashboard','crypto','cpp','continuous-delivery','command','cheatsheet',
          'certificate','business','build','bootstrap','awesome','audio','apple','apache','ansi','animation','alibaba','admin','activejob'
         ]



string_list = [['machinelearning-python','ml','machine-translation','machine-learning-from-scratch','machine-learning-algorithms','machine-learning'],['packages','package-manager','packaging'],['zsh-theme','zsh-prompt','zsh-configuration','zsh-autosuggestions'],['zero-copy','zero-configuration'],['youtube-live'],['xcode10'],['x86-64','x86'],['wysiwyg-editor'],
               ['workflow-automation','works-with-clojurescript','works-with-codespaces','works-with-construct','works-with-elm','works-with-flutter','works-with-gatsby','works-with-mint','works-with-phaser','works-with-react','works-with-reason','works-with-svelte','works-with-vue','works-with-yew'],
               ['windowing','windows-10','windows-console','windows-terminal','windowscontainers'],['weixin-plugin'],['wechat-macos','wechat-mini-program','wechat-plugin','wechat-weapp'],
               ['webxr','webworkers','webview','webtorrent','webstorm','websql','websockets','websocket-server','websocket-compression','websocket-client','websocket','website','webservices','webscraping','webrtc','webpack5','webpack4','webpack2','webpack','webp','webm','webgpu','webgl2','webgl','webframework','webfont','webfinger','webdriver','webdevelopment','webdav','webcomponents','webbrowser','webaudio','webassembly','webapp','webapi','web-terminal','web-server','web-performance','web-framework','web-development','web-components','web-clipper','web-audio','web-application-framework','web-application','web-app','web','sdwebimage','react-native-web','node-webkit','mobile-web','ipfs-web'],
               ['vuex','vuetifyjs','vuetify','vuepress','vuejs3','vuejs2','vuejs','vue3','vue2','vue-router','vue-material','vue-components','vue-component','vue-cli','vue-admin','ant-design-vue'],
               ['vpn-server','vpn-client'],['visual-studio','visualizer','visualizations','visualization','visual-studio-code-theme','visual-studio-code-extension','visual-studio-code','visual-studio','data-visualization'],
               ['vimrc','vimeo','vim-plugins','vim-plugin','vim-distribution','vim-configuration','vim'],['videojs','videocache','video-recording','video-player','video-conferencing','video-communication','video','scalable-video-conferences','html5-video','explanatory-math-videos','deep-video','3b1b-videos'],
               ['versioning','version-manager'],['utils','utility-classes','utilities'],['ui-library','ui-kit','ui-design','ui-components','redis-gui-client','gui-framework','gui-app'],
               ['typescript-playground','typescript-framework','typescript-definitions','typescript'],['twitter-streaming','twitter-cards','twitter','tweet'],
               ['touchscreen','touch-events','touch'],['unit-testing','tests','testing-tools','testing','test-suite','test-runners','test-runner','test-framework','test-automation','performance-testing','pentesting-windows','pentesting','pentesters','pentest','penetration-testing','painless-javascript-testing','load-testing','javascript-tests','interview-test','end-to-end-testing','e2e-tests','e2e-testing','api-testing','api-test'],
               ['windows-terminal','web-terminal','terminals','terminal-themes','terminal-schemes','terminal-emulators','terminal-color','terminal','osx-terminal-themes','lxterminal'],
               ['tensorflow-lite','tensorflow','tensor'],['telegram-solution','telegram-desktop'],['task-scheduler','task-runner','task-manager'],
               ['systems-science','systems','system-design','system','recommendedsystem','operating-system','management-system','fuse-filesystem','filesystem','embedded-systems','distributed-systems','design-systems','design-system','database-systems','content-management-system','build-system'],
               ['swiftyjson','swiftui','swiftpm','swift4-2','swift-programming','swift-package-manager','swift-library','swift-language','swift-framework','swift-extensions','swift','server-side-swift','rxswift','reactiveswift','openstack-swift','json-parsing-swift','ios-swift','awesome-swift','apple-swift'],
               ['swagger2','swagger-ui','swagger-js','swagger-api'],['svgo','svg-sprites','svg-path','svg-icons'],['stylesheets','stylesheet-language','styleguide','styled-components','style-linter','style-guide'],
               ['static-site-generator','static-code-analysis','static-analysis'],['statechart','state-tree','state-management','state-machine'],
               ['ssh-tunnel','ssh-server','ssh-agent'],['websql','sqlserver','sqlmap','sqlite','sql-server','sql-injection','sql-editor','sql','spark-sql','postgresql','nosql','mysql-compatibility','mysql','mssql','microsoft-sql-server'],
               ['springs','springmvc','springcloud-config','springcloud','springbootadmin','springboot-shiro','springboot','spring-session','spring-security','spring-physics','spring-framework','spring-data-jpa','spring-cloud-core','spring-cloud-alibaba','spring-cloud','spring-cache','spring-boot-upload-file','spring-boot-mongodb','spring-boot-mail','spring-boot-learning','spring-boot-examples','spring-boot-demo','spring-boot-2','spring-boot','spring','react-spring'],
               ['sorts','sorting-algorithms','sortable'],['software-engineering','software-engineer','software-development','software-analysis'],
               ['snippets-library','snippets-collection','snippets'],['websocket-server','web-server','vpn-server','ssh-server','sqlserver','sql-server','serverless-functions','serverless-framework','serverless-architectures','serverless','server-side-swift','server-side-rendering','server-rendering','server','parse-server','observer','microsoft-sql-server','jumpserver','http-server','graphql-server','dns-server','dhcp-server','dev-server','api-server'],
               ['spring-security','security-researchers','security-checklist','security-book','security','macos-security','macbook-security','docker-security','application-security'],
               ['site-search','silver-searcher','security-researchers','searches','search-in-text','search-engines','search-engine','search','research','jobsearch','jobs-search','hyperparameter-search','entreprise-search','elasticsearch','distributed-search-engine'],
               ['scikit-learn-python','scikit-learn'],['rfc-8259','rfc-7159','rfc-7049','rfc-6902','rfc-6901','rfc-6455'],['restful-api','restful','rest-api','rest','postgrest','api-rest','admin-on-rest'],
               ['redis-gui-client','redis-gui','redis-cluster','redis-client'],['works-with-react','reactjs','reactivex','reactiveui','reactiveswift','reactivecocoa','reactive-streams','reactive-programming','reactive','react16','react-tutorial','react-spring','react-router','react-navigation','react-native-web','react-native','react-interview-questions','react-hooks','react-components','react-component','react-boilerplate','react-apps','react-admin'],
               ['sqs-queue','queued-jobs','queue-workers','queue-tasks','queue','message-queue'],['pytorch-tutorial','pytorch-transformers'],['scikit-learn-python','python3','python-types','python-resources','python-requests','python-library','python-interview-questions','python-framework','python-cheatsheet','python-3','python','progress-bar-python'],
               ['protocol-compiler','protocol-buffers','protocol','protoc','ipfs-protocol','homekit-accessory-protocol','acme-protocol'],['progressmeter','progressbar','progress-bar-python','progress-bar'],['swift-programming','reactive-programming','programming-tutorial','programming-ligatures','programming-language','programming-interviews','programming-blogs','programming','programmer','javascript-programming','functional-programming','dynamic-programming-algorithm','dynamic-programming','concurrent-programming','competitive-programming'],
               ['posix-sh','posix-compliant-flags','posix-compliant'],['phpunit','phpstorm','phpmailer','php8','php7','php-library','php-installation','php-framework','php-applications','php','modern-php','designpatternsphp'],
               ['web-performance','performance-testing','performance-metrics','performance-analysis','performance','high-performance','best-performance'],
               ['openwrt-zh-cn','openwrt-shadowsocksr-libev-full','openwrt-package','openwrt-feed'],['notes','notebooks','notebook','note','jupyter-notebook','evernote'],['nodemon','nodejs-framework','nodejs-development','nodejs','node-webkit','node-module','node-js','node-framework','node'],
               ['neural-networks','neural-network','neural-nets','deep-neural-networks'],['natural-language-understanding','natural-language-processing','natural-language-generation','nlp-tasks','nlp-library'],['monitoring','monitor','error-monitoring','docker-monitoring'],
               ['polyglot-microservices','microservices-architecture','microservices'],['vue-material','training-materials','material-ui','material-theme','material-design-lite','material-design-for-bootstrap','material-design','material-components','material','bootstrap-material','angularjs-material'],
               ['wechat-macos','macosx','macos-setup','macos-security','macos-application'],['ubuntu-linux','linux-resource','linux-kernel','linux-insides','linux-desktop','linux-command','linux-app'],
               ['leetcode-solutions','leetcode-questions','leetcode-java','leetcode-golang','leetcode-go','leetcode-c'],['swift-language','stylesheet-language','ruby-language','programming-language','natural-language-understanding','natural-language-processing','natural-language-generation','multi-language','language-models','language-model','language-grammars','language-client','language','julia-language','esoteric-language'],
               ['kotlin-library','kotlin-android'],['ubjson','swiftyjson','json-serializer','json-serialization','json-schema','json-pointer','json-patch','json-parsing-swift','json-parsing-library','json-parser','json-merge-patch','json-diff','json','fastjson'],['painless-javascript-testing','javascript-tests','javascript-programming','javascript-motion-engine','javascript-modules','javascript-library','javascript-interview-questions','javascript-framework','javascript-engines','javascript-engine','javascript-compiler','javascript-closures','javascript-best-practices','javascript-applications','javascript-algorithms','javascript','es6-javascript','js-questions','js-ipfs','js'],
               ['ios-transition','ios-swift','ios-libraries','ios-lib','ios-framework','ios-animation'],['technical-coding-interview','react-interview-questions','rails-interview','python-interview-questions','programming-interviews','javascript-interview-questions','interviews','interviewing','interview-test','interview-questions','interview-preparation','interview-prep','interview-practice','interview','frontend-interview','front-end-interview','fe-interview','coding-interviews','coding-interview','angularjs-interview-questions','android-interview-questions'],
               ['httpurlresponse','https','httpie','httpclient','http2','http-server','http-proxy','http-client','http','automatic-https'],['htmlparser2','htmlparser','html5-video','html5-games','html5-game-development','html5-charts','html5-canvas','html5-boilerplate','html5-audio','html5','html-template','html-questions'],['headless-cms','headless-chrome','headless-browser'],
               ['graphql-server','graphql-relay','graphql-js','graphql-client','graphql-api','graphql','apollographql'],['hellogithub','gitops','gitlab','gitignore','github-pages','github-gist','github-api-v4','github-api','github-actions','github','gitea','gitbook','git'],['frontend-roadmap','frontend-interview','frontend-framework','frontend-development','frontend','front-end-interview','front-end-development','front-end-developer-tool','front-end'],
               ['webfont','variable-fonts','ttf-fonts','patched-fonts','opentype-fonts','iconic-fonts','icon-font','fonts','fontawesome','font-awesome'],['works-with-flutter','flutter-plugins','flutter-plugin','flutter-examples','flutter-doc','flutter-demo','flutter-apps'],['firefox-extension','firefox-addon'],['filesystem','files','file-uploads','file-uploader','file-upload','file-storage','file-sharing','file-manager','file-management'],
               ['openfaceswap','faceswap','face-swap','face-recognition','face-images','face-detection','deepfacelab','deepface','deep-face-swap'],['dockerfile','docker-ui','docker-swarm','docker-security','docker-registry','docker-monitoring','docker-machine','docker-image','docker-environment','docker-deployment','docker-container','docker-composer','docker-compose','docker-api'],['distributed-transactions','distributed-transaction','distributed-training','distributed-tracing','distributed-systems','distributed-search-engine','distributed-messaging','distributed-locks','distributed-database','distributed-configuration','distributed'],
               ['ui-design','system-design','material-design-lite','material-design-for-bootstrap','material-design','designpatternsphp','design-tools','design-systems','design-system','design-patterns-for-humans','design-patterns','design-pattern','design','api-design','ant-design-vue','ant-design'],['deepspeech','deeplearning','deepfakes','deepfacelab','deepface','deep-video','deep-reinforcement-learning','deep-neural-networks','deep-networks','deep-learning-tutorial','deep-learning','deep-io','deep-face-swap','creating-deepfakes'],['datasets','datamatrix','datamapper','datagrip','databricks','databases','database-systems','database-gui','database','data-viz','data-visualization','data-structures','data-structure','data-scientists','data-science','data-mining','data-mapper','data-flow','data-fetching','data-engineering','data-consistency','data-analytics','data-analysis'],
               ['functional-css','csse','css3-features','css3','css-tricks','css-reset','css-questions','css-in-js','css-framework','css-effects','css-animations'],['springcloud-config','springcloud','spring-cloud-core','spring-cloud-alibaba','spring-cloud','private-cloud','nextcloud','neteasecloud','google-cloud-storage','google-cloud-functions','cloudstorage','cloudnative','cloudmusic','cloudflare','cloud-storage','cloud-native','cloud-management','cloud-computing'],['headless-chrome','chrome-extensions','chrome-extension','chrome-devtools'],['zsh-autosuggestions','workflow-automation','test-automation','home-automation','build-automation','autosuggest','autopep8','autonomy','autonomous-vehicles','autonomous-driving','automl','automation','automatic-https','automatic-api','automated','autolayout','autograd','autocompletion','autocomplete','auto'],
               ['webscraping','webapi','swagger-api','strapi','scraping','restful-api','rest-api','public-apis','openapi3','openapi-specification','openapi','homekit-api','hapi','graphql-api','github-api-v4','github-api','fastapi','dom-apis','docker-api','automatic-api','apis','api-testing','api-test','api-server','api-rest','api-manager','api-management','api-gateway','api-documentation','api-design','api-client'],['angularjs-material','angularjs-interview-questions','angularjs','angular9','angular2','angular10','angular-dashboard-template','angular-components','angular-cli'],['mpandroidchart','kotlin-android','androidx','android-ui','android-studio','android-library','android-interview-questions','android-development','android-architecture'],
               ['sorting-algorithms','machine-learning-algorithms','javascript-algorithms','genetic-algorithm','dynamic-programming-algorithm','algorithms-implemented','algorithms-datastructures','algorithms-and-data-structures','algorithms','algorithm-competitions','algorithm-challenges'],
               ['pulltorefresh','pull-to-refresh','pull-request-review'],['public-trackers','public-tracker','public-key-pinning'],['pixiv','pixijs'],['openwrt','openvpn','openssl','opensource','openpose','opengl','opendata','opencv','openconnect','open-telemetry','open-source-project'],['objectstorage','objective-c-library','objective-c','object-storage','object-oriented'],['multithreading','multimedia','multiformats','multiboot','multi-platform','multi-person'],['miniprogram','minimalist','minikube','minify','minifier','minification','miniapp'],['low-code-plattform','low-code-development-platform','low-code'],
               ['localstack','localhost','localforage','local-development'],['loading','loaders','load-generator','load-balancer'],['kubernetes-ingress-controller','kubernetes-ingress'],['koa2','koa-middleware'],['imageloader','image-processor','image-processing','image-annotation'],['human-pose-estimation','human-pose','human-behavior-understanding','forhumans'],['puzzle-game','platform-game','not-the-game','games','gamedev','game-frameworks','game-engine','game-development','game-capture','game','facebook-instant-games'],['freeware-icon','freegate','freecodecamp','freebsd-vt','free-ssr','free-ss','free-software','free-fanqiang'],['object-detection','keypoint-detection','intrusion-detection','feature-detection'],['dragging','draggable','drag-drop','drag-and-drop'],['documentation-tool','documentation'],['devops-roadmap'],
                ['hystrix-dashboard','dashboards','dashboard-templates','dashboard','admin-dashboard'],['cryptography','cryptocurrency'],['cpplint','cppcon','cpp11','cpp-library'],['continuous-integration','continuous-deployment'],['subcommands','commandline','command-line-tool','command-line','command-cobra'],['cheatsheets'],
               ['certification','certificates','certificate-pinning'],['businessintelligence','business-intelligence','business-analytics'],['form-builder','buildpack','build-tools','build-tool','build-pipelines'],['secure-boot','multiboot','jeecgboot','jeecg-boot','bootstrap5','bootstrap4-theme','bootstrap4','bootstrap','bootable-usb'],['awesomeness','awesome-public-datasets','awesome-lists','awesome-list'],['audio-processing','audio-library'],['apple-watch','apple-tv','apple'],['apache-superset','apache-airflow'],['strip-ansi','ansible','ansi-escape-codes','ansi-colors','ansi'],['transition-animation','recyclerview-item-animation','animations','animation-library'],['alibaba-oss','alibaba-middleware'],['ngx-admin','ng2-admin','eladmin','adminpanel','admin-ui','admin-template','admin-panel','admin-dashboard'],
               ['activerecord','activemq','activejob','active-record']
                
              ]
def same_things(string):
    for idx,i in enumerate(string_list):
        if string in i:
            string = target[idx]    
    return string

In [19]:
# 전체 패키지들이 어떤 토픽들을 가지고 있는지 누적합을 딕셔너리 형태로 만듬
topic_dic = {}
for idx,j in enumerate(df_topic2['topic_keyword']):
    for i in j:
        i = i.strip()
        i = same_things(i) # 위에서 만든 여러 토픽을 하나의 토픽으로 묶는 함수
        if len(i)==0:
            continue
        if i not in topic_dic:
            topic_dic[i] = 1
        else:
            topic_dic[i] +=1
result = []
topic_dic = sorted(topic_dic.items(), key=lambda x: x[1], reverse=True)

topic_list = [i[0] for i in topic_dic]
# 위 과정들을 통해 얻은 토픽들을 통해 칼럼을 만듬
df_vector = pd.DataFrame(topic_list).transpose()

topic_dic_df = {}
for i in topic_list:
    topic_dic_df[i]=0
# 토픽들을 칼럼으로 가지는 각각의 패키지 별로 벡터를 만들고 모든 성분을 0으로 채움
df_vector = pd.DataFrame(result,index=df_topic2['project_name'],columns=topic_list).fillna(0)

# 각 패키지 별로 가지고있는 토픽을 +1 씩 하여 각 패키지의 벡터를 생성함
for idx,topics in enumerate(df_topic2['topic_keyword']):
    for topic in topics:
        if topic in topic_list:
            topic = same_things(topic)
            df_vector.loc[df_topic2['project_name'].iloc[idx],topic] += 1
df_vector.to_excel("StarsOver{}_vectors_상위{}페이지.xlsx".format(stars,pages),index=False)
print("StarsOver{}_vectors_상위{}페이지.xlsx".format(stars,pages),"저장완료.\n")



# print("{} {} DBSCAN 클러스터링 시작 {}".format("*"*10,o,"*"*10))
excel_name = "StarsOver{}_vectors_상위{}페이지.xlsx".format(stars,pages)

df_vector = pd.read_excel(excel_name)
# eps 값을 조정해나가면서 클러스터링을 해야 더 정확한 결과가 나온다
dbscan = DBSCAN(eps = 0.3)
dbscan_cluster = dbscan.fit_predict(df_vector)
dbscan_cluster
dbscan_clustered_dic = {}
dbscan_clustered_list = []
dbscan_cluster_num = len(set(dbscan_cluster))


for idx,i in enumerate(dbscan_cluster):
    if i not in dbscan_clustered_dic:
        dbscan_clustered_dic[i] = [df_topic2['project_name'][idx]]
    else:
        dbscan_clustered_dic[i].append(df_topic2['project_name'][idx])

# 클러스터링이 된 패키지들
# 20개의 군집으로 생성

dbscan_clustered_dic = sorted(dbscan_clustered_dic.items(), key=lambda x: x[0])

df_dbscan_cluster = pd.DataFrame(dbscan_clustered_dic,columns=['num','clusters'])
dbscan_cluster_num = [len(i) for i in df_dbscan_cluster['clusters']]
df_dbscan_cluster['cluster_num'] = dbscan_cluster_num
topic_dbscan_clustered_list = []
for i in df_dbscan_cluster['clusters']:
    temp_dic = {}
    for j in i:
        topics = df_topic2[df_topic2['project_name']==j]['topic_keyword'].values[0]
        for i in topics:
            if len(i)==0:
                continue
            i = same_things(i)
            if i not in temp_dic:
                temp_dic[i] = 1
            else:
                temp_dic[i] += 1
    temp_dic = sorted(temp_dic.items(), key=lambda x: x[1], reverse=True)
    topic_dbscan_clustered_list.append(temp_dic[:-1])
df_dbscan_cluster['topics'] = topic_dbscan_clustered_list
df_dbscan_cluster.to_excel("StarsOver{}_상위{}페이지_DBSCAN_clusters.xlsx".format(stars,pages),index=False)
print(df_dbscan_cluster)
print("StarsOver{}_상위{}페이지_DBSCAN_clusters.xlsx".format(stars,pages),"저장완료")
print("*"*50)


react
nodejs
javascript
d3
teachers
community
education
programming
curriculum
math
learn-to-code
nonprofits
careers
hacktoberfest
education
list
books
resource
hacktoberfest
computer-science
algorithm
interview
study-plan
lists
awesome
unicorns
resources
computer-science
engineering
roadmap
study-plan
python-roadmap
developer-roadmap
react-roadmap
java-roadmap
backend-roadmap
go-roadmap
angular-roadmap
dba-roadmap
api
lists
open-source
list
development
public
resources
dataset
free
software
public-api
javascript
framework
vue
frontend
react
javascript
library
ui
frontend
declarative
python
design
development
programming
web
system
interview
python
deep-learning
tensorflow
distributed
javascript
css
html
sass
bootstrap
scss
javascript
education
programming
es6
async
book
book-series
es2015
learn-to-code
training-providers
closures
prototypes
python
java
computer-science
algorithm
leetcode
cpp
interview
programming
tutorials
free
tutorial-code
tutorial-exercises
shell
theme
zsh
cli
prod

In [21]:
df_topic2

project_name  \
0                freeCodeCamp/freeCodeCamp   
1                           996icu/996.ICU   
2   EbookFoundation/free-programming-books   
3      jwasham/coding-interview-university   
4                     sindresorhus/awesome   
5          kamranahmedse/developer-roadmap   
6                  public-apis/public-apis   
7                                vuejs/vue   
8                           facebook/react   
9         donnemartin/system-design-primer   
10                   tensorflow/tensorflow   
11                          twbs/bootstrap   
12                 getify/You-Dont-Know-JS   
13                        CyC2018/CS-Notes   
14        codecrafters-io/build-your-own-x   
15                         ohmyzsh/ohmyzsh   
16          trekhleb/javascript-algorithms   
17                         flutter/flutter   
18                    TheAlgorithms/Python   
19                        github/gitignore   
20                          torvalds/linux   
21                        microsoft/vscode   
22                    vinta/awesome-python   
23                       airbnb/javascript   
24                    Snailclimb/JavaGuide   
25               jackfrued/Python-100-Days   
26                   ossu/computer-science   
27                     ytdl-org/youtube-dl   
28            labuladong/fucking-algorithm   
29           jlevy/the-art-of-command-line   

                                        topic_keyword star_number  
0   [react, nodejs, javascript, d3, teachers, comm...        348k  
1                                                  []        263k  
2   [education, list, books, resource, hacktoberfest]        238k  
3   [computer-science, algorithm, algorithms, inte...        223k  
4   [lists, awesome, unicorns, resources, awesome-...        207k  
5   [computer-science, engineering, roadmap, study...        199k  
6   [api, lists, open-source, list, development, p...        199k  
7              [javascript, framework, vue, frontend]        197k  
8   [react, javascript, library, ui, frontend, dec...        191k  
9   [python, design, development, programming, web...        187k  
10  [python, machine-learning, deep-neural-network...        166k  
11  [javascript, css, html, sass, bootstrap, scss,...        158k  
12  [javascript, education, programming, es6, asyn...        156k  
13  [python, java, computer-science, algorithm, le...        154k  
14  [programming, tutorials, free, awesome-list, t...        147k  
15  [shell, theme, zsh, cli, productivity, oh-my-z...        147k  
16  [javascript, computer-science, algorithm, algo...        145k  
17  [android, windows, macos, dart, ios, mobile, w...        142k  
18  [python, education, algorithm, practice, inter...        140k  
19                                   [gitignore, git]        135k  
20                                                 []        134k  
21  [electron, microsoft, editor, typescript, visu...        134k  
22  [python, awesome, python-library, collections,...        132k  
23  [javascript, styleguide, eslint, es6, style-li...        124k  
24  [mysql, java, redis, spring, system, algorithm...        124k  
25                                                 []        120k  
26  [computer-science, curriculum, courses, awesom...        119k  
27                                                 []        111k  
28  [computer-science, algorithms, leetcode, data-...        108k  
29  [windows, macos, linux, bash, documentation, u...        107k